#### What are you trying to do in this notebook?
In this competition, I’ll identify elements in student writing. More specifically, I will automatically segment texts and classify argumentative and rhetorical elements in essays written by 6th-12th grade students. I'll access to the largest dataset of student writing ever released in order to test your skills in natural language processing, a fast-growing area of data science.

#### Why are you trying it?
I'll make it easier for students to receive feedback on their writing and increase opportunities to improve writing outcomes. Virtual writing tutors and automated writing systems can leverage these algorithms while teachers may use them to reduce grading time. The open-sourced algorithms you come up with will allow any educational organization to better help young writers develop.

My task is to predict the human annotations. I will first need to segment each essay into discrete rhetorical and argumentative elements (i.e., discourse elements) and then classify each element as one of the following:

- Lead - an introduction that begins with a statistic, a quotation, a description, or some other device to grab the reader’s attention and point toward the thesis

- Position - an opinion or conclusion on the main question

- Claim - a claim that supports the position

- Counterclaim - a claim that refutes another claim or gives an opposing reason to the position

- Rebuttal - a claim that refutes a counterclaim

- Evidence - ideas or examples that support claims, counterclaims, or rebuttals.

- Concluding Statement - a concluding statement that restates the claims

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# libraries
import os
import numpy as np 
import pandas as pd 
pd.options.mode.chained_assignment = None
from tqdm.notebook import tqdm
tqdm.pandas()
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# nltk
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer

In [ ]:
# files
os.listdir("../input/feedback-prize-2021")

In [ ]:
submission = pd.read_csv("../input/feedback-prize-2021/sample_submission.csv")
submission.head()

In [ ]:
train_df = pd.read_csv('../input/feedback-prize-2021/train.csv')
train_df.head()

In [ ]:
train_df.discourse_type.value_counts(normalize=False)

In [ ]:
train_dir = "../input/feedback-prize-2021/train"
test_dir = "../input/feedback-prize-2021/test"

In [ ]:
def create_full_text_dataframe(train=True) -> pd.DataFrame:
    id_list = []
    text_list = []
    
    if train:
        for id in tqdm(train_df.id):
            filepath = os.path.join(train_dir, f"{id}.txt")
            text = open(filepath, 'r').read()

            id_list.append(id)
            text_list.append(text)
    else:
        for filename in tqdm(os.listdir(test_dir)):
            filepath = os.path.join(test_dir, filename)
            id = str(filename).strip()[:-4]
            text = open(filepath, 'r').read()
            
            id_list.append(id)
            text_list.append(text)
            
    return pd.DataFrame(data={"id": id_list, "text": text_list})

In [ ]:
df_train = create_full_text_dataframe()

In [ ]:
df_test = create_full_text_dataframe(train=False)

In [ ]:
df_train = df_train.merge(train_df, on="id", how="inner")

In [ ]:
df = df_train[["id", "discourse_text", "discourse_type"]]

In [ ]:
# lemmatize (or stem?)

#lemmatizer = WordNetLemmatizer()
#df['discourse_text'] = df['discourse_text'].progress_apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)]))

In [ ]:
# Target label encoding
encoder = LabelEncoder()
labels = encoder.fit_transform(df.discourse_type)

# encoded target labels
df.loc[:, "label"] = labels

In [ ]:
# train/test splitting
X_train, X_test, y_train, y_test = train_test_split(df.discourse_text, df.label, test_size=0.02)

In [ ]:
# Model Pipeline and training

multinomialNB = Pipeline([
        ('vect', CountVectorizer(stop_words='english', ngram_range=(1, 2), binary=True)),
        ('tfidf', TfidfTransformer(norm='l2', use_idf=True)),
        ('clf', MultinomialNB(alpha=0.1)),    
])

# training
multinomialNB.fit(X_train, y_train)

In [ ]:
# prediction on test data
y_test_pred = multinomialNB.predict(X_test)

In [ ]:
# Results on test set
print("\nTest Precision:", metrics.precision_score(y_test, y_test_pred, average='micro'))
print("\nTest Recall:", metrics.recall_score(y_test, y_test_pred, average='micro'))
print("\nClassification Report:\n", metrics.classification_report(y_test, y_test_pred))

In [ ]:
def plot_confusion_matrix(y_test, y_scores, class_names):
    num_class = len(class_names)
    cm = metrics.confusion_matrix(y_test, y_scores)

    # normalize
    con = np.zeros((num_class, num_class))
    for x in range(num_class):
        for y in range(num_class):
            con[x,y] = cm[x,y]/np.sum(cm[x,:])

    plt.figure(figsize=(10,8))
    sns.set(font_scale=1.0) # for label size
    sns.heatmap(con, annot=True, fmt=".2", cmap='Blues',xticklabels= class_names , yticklabels= class_names)
    plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_test_pred, encoder.inverse_transform(np.unique(labels)))

In [ ]:
# create df_test: a record from each discourse within test text docs
def expand_df_test(df: pd.DataFrame=df_test) -> pd.DataFrame:
    
    ids = []; data = []
    for id, text in zip(df.id, df.text):
        sentences = nltk.sent_tokenize(text)
        
        id_sentences = []; idx = 0 
        for sentence in sentences:
            id_sentence = []
            words = sentence.split()
            for w in words:
                id_sentence.append(idx)
                idx += 1
            id_sentences.append(id_sentence)
        data += list(zip([id] * len(sentences), sentences, id_sentences))
        
    tmp = pd.DataFrame(data, columns=['id', 'text', 'predictionstring'])
    return tmp

In [ ]:
test = expand_df_test()

In [ ]:
# prediction on test data

test["class"] = multinomialNB.predict(test.text)

In [ ]:
test.loc[:, "class"] = encoder.inverse_transform(test["class"].values)

In [ ]:
# submission
result = test[["id", "class", "predictionstring"]]
result['predictionstring'] = result['predictionstring'].apply(lambda x: ' '.join([str(i) for i in x]))

In [ ]:
result.head()

In [ ]:
result.to_csv("submission.csv", index=False)

#### Did it work?
I'll make it easier for students to receive feedback on their writing and increase opportunities to improve writing outcomes. Virtual writing tutors and automated writing systems can leverage these algorithms while teachers may use them to reduce grading time. The open-sourced algorithms you come up with will allow any educational organization to better help young writers develop.

#### What did you not understand about this process?
Well, everything provides in the competition data page. I've no problem while working on it. If you guys don't understand the thing that I'll do in this notebook then please comment on this notebook.

#### What else do you think you can try as part of this approach?
Writing is a critical skill for success. However, less than a third of high school seniors are proficient writers, according to the National Assessment of Educational Progress. Unfortunately, low-income, Black, and Hispanic students fare even worse, with less than 15 percent demonstrating writing proficiency. One way to help students improve their writing is via automated feedback tools, which evaluate student writing and provide personalized feedback.